In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import scale
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import random
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tag import StanfordNERTagger
from nltk.tokenize import sent_tokenize
from nltk.tokenize import sent_tokenize


import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import scale
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from nltk import word_tokenize, pos_tag
from nltk.tag import StanfordNERTagger
from sklearn.linear_model import RidgeClassifier
# from tweetokenize import Tokenizer


import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import normalize
from sklearn import preprocessing
from nltk.tag.stanford import StanfordNERTagger
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer


In [2]:
venezuela_tweets = pd.read_csv('/Users/alessandro/Downloads/venezuela_201906_1_tweets_csv_hashed.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (19,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
venezuela_tweets.columns

Index(['tweetid', 'userid', 'user_display_name', 'user_screen_name',
       'user_reported_location', 'user_profile_description',
       'user_profile_url', 'follower_count', 'following_count',
       'account_creation_date', 'account_language', 'tweet_language',
       'tweet_text', 'tweet_time', 'tweet_client_name', 'in_reply_to_userid',
       'in_reply_to_tweetid', 'quoted_tweet_tweetid', 'is_retweet',
       'retweet_userid', 'retweet_tweetid', 'latitude', 'longitude',
       'quote_count', 'reply_count', 'like_count', 'retweet_count', 'hashtags',
       'urls', 'user_mentions', 'poll_choices'],
      dtype='object')

In [4]:
venezuela_tweets.shape

(569455, 31)

In [5]:
venezuela_tweets.userid.value_counts()

568620329                                       43231
801485278858854401                              39421
801200139155214336                              34912
801555575687495681                              33503
801463213833547776                              32870
751630413769113601                              31657
751635009052827648                              31073
881675624401833984                              27436
761401522798333952                              27209
892133369965944833                              26370
769380097359024128                              26136
768237282562310144                              26020
896121971989262336                              25472
778257194076831744                              22624
741092772233482241                              16400
801221525718335488                              12618
772513370599747585                              12099
801442896041873409                              12025
741112500356276225          

# Only English Words (so it doesn't crash my notebook)

In [6]:
tweets_mask = venezuela_tweets['is_retweet'] == False
tweets = venezuela_tweets[tweets_mask]

tweets_english = tweets[tweets['tweet_language'] == 'en']

In [7]:
data = tweets_english[['user_screen_name','tweet_text']]


In [8]:
data.user_screen_name

0                                       TRUMPTRAIN_17
1                                       Laureen4Trump
2                                       Laureen4Trump
3                                        EmmaTurnerBN
4                                      GODBLESSAMERIC
5                                      GODBLESSAMERIC
6                                      GODBLESSAMERIC
7                                      GODBLESSAMERIC
8                                      GODBLESSAMERIC
9                                      GODBLESSAMERIC
10                                     GODBLESSAMERIC
11                                     GODBLESSAMERIC
12                                    TrumpNewsDaily_
13                                    TrumpNewsDaily_
14                                    TrumpNewsDaily_
15                                    TrumpNewsDaily_
16                                    TrumpNewsDaily_
17                                    TrumpNewsDaily_
38                          

In [9]:
# data = data[:10000]

In [10]:
# len(data)

### Creating a function to clean_up the twitter_text
#### ALL WORKS!!!!
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

#Additional
import string
string.punctuation

# In this edit I didn't remove # and @ hoping to find the way to remove it together with the followings
punct = set(string.punctuation)
punct.remove('@')
punct.remove('#')


lemmer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

import preprocessor as p
import re

def preprocessing_text(text):
    '''
    INPUT: str
    OUTPUT: str w/ emojies, urls, hashtags and mentions removed
    '''
    p.set_options(p.OPT.EMOJI, p.OPT.URL, p.OPT.HASHTAG, p.OPT.MENTION, p.OPT.NUMBER)
    clean_text = p.clean(text)
    
    return clean_text

def remove_symbols(word, symbol_set):
    
    '''
    Removing symbols from word
    '''
    return ''.join(char for char in word 
                    if char not in symbol_set)
def clean_tweet_text(text_column):
    '''
    takes a columns in dataframe with tweets text: 
    Outputs: PD Series of tokenized docs
    lower case, 
    symbol_set charachters removed
    punctuation removed
    words stemmed and lemmatized
    
    '''
    
    
    # converting from pd to list
    corpus = text_column.values.tolist()
    
    #Removing all HTTPs
    docs_no_http = [ re.sub(r'https?:\/\/.*\/\w*', '', doc) for doc in corpus ]
    #First ---> tokenize docs
    tokenized_docs = [doc.split() for doc in docs_no_http]
    # Lower case words in doc
    tokenized_docs_lowered  = [[word.lower() for word in doc]
                                for doc in tokenized_docs]

    # Removing punctuation from docs
    cleaned_docs = [[remove_symbols(word, punct) for word in doc] 
                    for doc in tokenized_docs_lowered]

    ### Removing stop words
#     stop_words = set(nltk.corpus.stopwords.words('english'))
#     docs_no_stops = [[word for word in doc if word not in stop_words] 
#                      for doc in cleaned_docs]
    # Lemmatize words in docs
    docs_lemmatized = [[lemmer.lemmatize(word) for word in doc]
                      for doc in docs_no_stops]
    
    # Stem words in docs
    docs_stemmed = [[stemmer.stem(word) for word in doc]
                      for doc in docs_lemmatized]
    
    # Removes mentions, emotions, hashtags and emojies
    docs_no_mentions = [preprocessing_text(' '.join(doc)) for doc in docs_stemmed]
    
    bag = []
    for doc in docs_no_mentions:
        if len(doc) >= 2:
            bag.append(doc)
    
    # converts into list of lists
    bow = [list(tweet.split(' ')) for tweet in bag]
    
    
    # convert docs into pd series
    full_service_docs_series = pd.Series( (v[0] for v in bow) )
    
    return bag, bow, docs_stemmed, full_service_docs_series

# Corpus and Users built to loop through. Corpus will be X and Users Y.

In [11]:
def get_bag_of_words(tweet):
    tokenized = [ word for word in tweet.split() if not(word.startswith("http")) and not(word.startswith('#'))]
    return ' '.join(tokenized)



In [12]:
### Creating a function to clean_up the twitter_text
#### ALL WORKS!!!!
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

#Additional
import string
string.punctuation

# In this edit I didn't remove # and @ hoping to find the way to remove it together with the followings
punct = set(string.punctuation)
punct.remove('@')
punct.remove('#')


lemmer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

import preprocessor as p
import re

def preprocessing_text(text):
    '''
    INPUT: str
    OUTPUT: str w/ emojies, urls, hashtags and mentions removed
    '''
    p.set_options(p.OPT.EMOJI, p.OPT.URL, p.OPT.HASHTAG, p.OPT.MENTION, p.OPT.NUMBER)
    clean_text = p.clean(text)
    
    return clean_text

def remove_symbols(word, symbol_set):
    
    '''
    Removing symbols from word
    '''
    return ''.join(char for char in word 
                    if char not in symbol_set)
def clean_tweet_text(text_column):
    '''
    takes a columns in dataframe with tweets text: 
    Outputs: PD Series of tokenized docs
    lower case, 
    symbol_set charachters removed
    punctuation removed
    words stemmed and lemmatized
    
    '''
    
    
    # converting from pd to list
    corpus = text_column.values.tolist()
    
    #Removing all HTTPs
    docs_no_http = [ re.sub(r'https?:\/\/.*\/\w*', '', doc) for doc in corpus ]
    #First ---> tokenize docs
    tokenized_docs = [doc.split() for doc in docs_no_http]
    # Lower case words in doc
    tokenized_docs_lowered  = [[word.lower() for word in doc]
                                for doc in tokenized_docs]

    # Removing punctuation from docs
    cleaned_docs = [[remove_symbols(word, punct) for word in doc] 
                    for doc in tokenized_docs_lowered]

    ### Removing stop words
#     stop_words = set(nltk.corpus.stopwords.words('english'))
#     docs_no_stops = [[word for word in doc if word not in stop_words] 
#                      for doc in cleaned_docs]
    # Lemmatize words in docs
    docs_lemmatized = [[lemmer.lemmatize(word) for word in doc]
                      for doc in docs_no_stops]
    
    # Stem words in docs
    docs_stemmed = [[stemmer.stem(word) for word in doc]
                      for doc in docs_lemmatized]
    
    # Removes mentions, emotions, hashtags and emojies
    docs_no_mentions = [preprocessing_text(' '.join(doc)) for doc in docs_stemmed]
    
    bag = []
    for doc in docs_no_mentions:
        if len(doc) >= 2:
            bag.append(doc)
    
    # converts into list of lists
    bow = [list(tweet.split(' ')) for tweet in bag]
    
    
    # convert docs into pd series
    full_service_docs_series = pd.Series( (v[0] for v in bow) )
    
    return bag, bow, docs_stemmed, full_service_docs_series

In [13]:
data['bag_of_words'] = data['tweet_text'].apply(get_bag_of_words)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
data.head()

,user_screen_name,tweet_text,bag_of_words
0,TRUMPTRAIN_17,Truth About “Right to Healthcare” Every Lib Ne...,truth about “right to healthcare” every lib ne...
1,Laureen4Trump,"BREAKING: Air Force Makes Tragic Announcement,...",breaking air force makes tragic announcement p...
2,Laureen4Trump,HAPPENING NOW: Disney Facing a Scandal So Cata...,happening now disney facing a scandal so catas...
3,EmmaTurnerBN,Another Stupid Gun Control Cartoon From the Le...,another stupid gun control cartoon from the left
4,GODBLESSAMERIC,➡️ 90 POUNDS OF COCAINE FOUND ON SHIP OWNED BY...,➡️ 90 pounds of cocaine found on ship owned by...


In [22]:
import string

string.punctuation

punct = set(string.punctuation)
punct.remove('@')
punct.remove('#')


lemmer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

data['bag_of_words'] = data['bag_of_words'].apply(lambda x: x.lower())
data['bag_of_words'] = data['bag_of_words'].apply(lambda x: remove_symbols(x, punct))
data['bag_of_words'] = data['bag_of_words'].apply(lambda x: re.sub(r'https?:\/\/.*\/\w*', '', x))
data['bag_of_words'] = data['bag_of_words'].apply(lambda x: lemmer.lemmatize(x))
data['bag_of_words'] = data['bag_of_words'].apply(lambda x: stemmer.stem(x))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

# User Df's

# 1 Evelyn_df

In [16]:
Evelyn_df = data[data.user_screen_name == 'EvelynWhiteGOP']


# 2 OliviaAllenC

In [17]:
OliviaAllenC = data[data.user_screen_name == 'OliviaAllenC']


# TrumpNewz

In [18]:
TrumpNewsz= data[data.user_screen_name == 'TrumpNewsz']

# comparing TrumpNewz to OliviaAllenC (similar size)

In [19]:
fletchers_home = pd.concat([Evelyn_df, TrumpNewsz])
fletchers_home.user_screen_name.value_counts()

EvelynWhiteGOP    38879
TrumpNewsz        34665
Name: user_screen_name, dtype: int64

In [20]:
X = document_tfidf_matrix
y = fletchers_home.user_screen_name

NameError: name 'document_tfidf_matrix' is not defined

In [ ]:
tfidf = TfidfVectorizer(decode_error='replace', max_features=150)
document_tfidf_matrix = tfidf.fit_transform(fletchers_home.bag_of_words)
# print(sorted(tfidf.vocabulary_))
# print(document_tfidf_matrix.todense())

In [ ]:
first do train test split and then vectorize it

In [ ]:
X = document_tfidf_matrix
y = fletchers_home.user_screen_name
print(X.shape)
print(y.shape)


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size =.3)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=300, max_depth=2,
                             random_state=42)
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
# confusion matrix

In [ ]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
     max_depth=1, random_state=0).fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)  

# Olivia allen v trumpnews count vectorizer

In [ ]:
matrix = CountVectorizer(max_features=150)

In [ ]:
X = matrix.fit_transform(fletchers_home.bag_of_words).toarray()
y = fletchers_home.user_screen_name


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size =.3)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=300, max_depth=2,
                             random_state=42)
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
     max_depth=1, random_state=0).fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)  

In [ ]:
rfc=RandomForestClassifier()


CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

In [ ]:
CV_rfc.score(X_test, y_test)  

# TrumpNewsDaily

# comparing Trump news

In [23]:

TrumpNewsDaily_= data[data.user_screen_name == 'TrumpNewsDaily_']

In [24]:
trump_news_comparison = pd.concat([TrumpNewsDaily_, TrumpNewsz])
trump_news_comparison.user_screen_name.value_counts()

TrumpNewsz         34665
TrumpNewsDaily_    32407
Name: user_screen_name, dtype: int64

In [ ]:
# tfidf = TfidfVectorizer(decode_error='replace', max_features=150)
# document_tfidf_matrix = tfidf.fit_transform(trump_news_comparison.bag_of_words)

In [49]:
X = trump_news_comparison.bag_of_words
y = trump_news_comparison.user_screen_name

In [50]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size =.3)

In [51]:
tfidf = TfidfVectorizer(decode_error='replace', max_features=150)
X_train = tfidf.fit_transform(X_train)

In [52]:
X_train

<46950x150 sparse matrix of type '<class 'numpy.float64'>'
	with 229131 stored elements in Compressed Sparse Row format>

In [53]:
X_test = tfidf.fit_transform(X_test)

In [54]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=300, max_depth=2,
                             random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [55]:
y_pred = rf.predict(X_test)

In [56]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.504770897525097

In [29]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
     max_depth=1, random_state=0).fit(X_train, y_train)

NameError: name 'GradientBoostingClassifier' is not defined

In [ ]:
clf.score(X_test, y_test)  

In [ ]:
# try Lauren and Lauren

# 3.LaurenJonesGOP_  

In [ ]:
linear regression, decison tree, latent dirichlet allocation, 

In [ ]:
LaurenJonesGOP = data[data.user_screen_name == 'LaurenJonesGOP_']


# 4.SophiaMillerC

In [ ]:
SophiaMillerC = data[data.user_screen_name == 'SophiaMillerC']

# 5. SamanthaClarkH

In [ ]:
 SamanthaClarkH = data[data.user_screen_name == 'SamanthaClarkH']

# 6. Sophia4Trump

In [ ]:
 Sophia4Trump = data[data.user_screen_name == 'Sophia4Trump']

# 7. AlyssaNelsonR

# Count Vectorizer

In [ ]:
matrix = CountVectorizer(max_features=150)

In [ ]:
X = matrix.fit_transform(data.tweet_text).toarray()

# The TFIDF vector

# Run count vec instead?

In [ ]:
tfidf = TfidfVectorizer(decode_error='replace', max_features=150)
document_tfidf_matrix = tfidf.fit_transform(data.bag_of_words)
# print(sorted(tfidf.vocabulary_))
print(document_tfidf_matrix.todense())

In [ ]:
document_tfidf_matrix.todense().shape

In [ ]:
# df = pd.DataFrame(document_tfidf_matrix.todense(),columns=tfidf.vocabulary_)
# df.head()

In [ ]:
# X = document_tfidf_matrix
y = data.user_screen_name
print(X.shape)
print(y.shape)


In [ ]:
# X[:3,:10]

# This kernel dies which makes it so  i have to restart everything


In [ ]:


# This kernel dies which makes it so everything i have to restart everything
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size =.3)





# Try RandomForest

# grid search on n_estimators, max_depth :10,200,300 and then decrease max depth

# sklearn ensemble gradient boost, grid search: max depth, max features, learning rate

# SVM 

# stop words, count vec. 
# take stop words and look at distribution. count. using that as additional feature. 
# take original x. append that to your X. 
# libraries out there to use parts of speech tagging.

# Hamid Advice: Binary pick two users for tfidf for 2 user.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=300, max_depth=2,
                             random_state=42)
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
(y_test == y_pred).mean()

In [ ]:
precision_recall_fscore_support

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report, precision_recall_fscore_support, accuracy_score
precision_recall_fscore_support(y_test, y_pred, average='macro')

In [ ]:
precision_recall_fscore_support()

In [ ]:
accuracy = accuracy_score(y_test, y_pred)


In [ ]:
accuracy

In [ ]:
cr = classification_report(y_test, y_pred)
cr

In [ ]:
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [2,6],
    'criterion' :['gini', 'entropy']
}

In [ ]:
rfc=RandomForestClassifier()


In [ ]:
rfc=RandomForestClassifier()


CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

# Gradient Booster

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier


In [ ]:
from sklearn.model_selection import GridSearchCV


In [ ]:
run it in atom ok

In [ ]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
     max_depth=1, random_state=0).fit(X_train, y_train)
# clf.score(X_test, y_test)  

In [ ]:
clf.score(X_test, y_test)  

In [ ]:
# Grid Search Parameters
parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10,100,1000]
    }

In [ ]:
# sklearn ensemble gradient boost, grid search: max depth, max features, learning rate

In [ ]:
clf = GridSearchCV(GradientBoostingClassifier(), parameters, cv=3, n_jobs=-1)


In [ ]:
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.best_params_)

# Naive Bayes

In [ ]:
# Naive Bayes 
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predict Class
y_pred = classifier.predict(X_test)

# Accuracy 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
accuracy

# Logistic Regression Classifier

In [ ]:
lr = LogisticRegressionCV(cv=3, random_state=0, multi_class='multinomial').fit(X_train, y_train)
lr.scores_

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
(y_pred == y_test).mean()

In [ ]:
print(classification_report(y_test,y_pred))

# Pre processing pipeline

In [ ]:
import string

string.punctuation

punct = set(string.punctuation)
punct.remove('@')
punct.remove('#')


lemmer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

data['tweet_text'] = data['tweet_text'].apply(lambda x: x.lower())
data['tweet_text'] = data['tweet_text'].apply(lambda x: remove_symbols(x, punct))
data['tweet_text'] = data['tweet_text'].apply(lambda x: re.sub(r'https?:\/\/.*\/\w*', '', x))
data['tweet_text'] = data['tweet_text'].apply(lambda x: lemmer.lemmatize(x))
data['tweet_text'] = data['tweet_text'].apply(lambda x: stemmer.stem(x))

# Truncated SVD

# This kernel dies which makes it so everything i have to restart everything

In [ ]:
from sklearn.decomposition import TruncatedSVD

# X = document_tfidf_matrix.toarray()
# svd = TruncatedSVD(n_components=3000, random_state=42)
# svd.fit(X)

Go back to lecture read everything.


Realistic Braulin

